In [1]:
pip install tensorflow


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
pip install -U pycryptodome

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [61]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

In [62]:
dataset = tf.keras.preprocessing.image_dataset_from_directory("Dataset_T", shuffle = True, image_size  = (256,256), batch_size =32)

Found 4500 files belonging to 3 classes.


In [63]:
class_names = dataset.class_names
class_names

['Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_healthy']

In [65]:
dataset = dataset.shuffle(10000, seed = 12)

In [66]:
train_ds = dataset.take(110)
len(train_ds)

110

In [67]:
left_ds= dataset.skip(110)
len(left_ds)

31

In [68]:
validation_ds = left_ds.take(15)
len(validation_ds)

15

In [69]:
test_ds= left_ds.skip(15)
len(test_ds)

16

In [70]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = validation_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [71]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(256,256),
    layers.experimental.preprocessing.Rescaling(1.0/255),
])

In [72]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.2),
])

In [73]:
tomato_model = models.Sequential([
    resize_and_rescale,
    data_augmentation,
    
    layers.Conv2D(32, (3,3), activation ='relu', input_shape = (32,256,256,3), padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
#     layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
#     layers.MaxPooling2D(2,2),
#     layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
#     layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation ='relu',padding = "same"),
    layers.MaxPooling2D(2,2),
    
    layers.Flatten(),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(3, activation = 'softmax')
])

tomato_model.build(input_shape = (32,256,256,3))

In [74]:
tomato_model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_12 (Sequential)  (32, 256, 256, 3)         0         
                                                                 
 sequential_13 (Sequential)  (32, 256, 256, 3)         0         
                                                                 
 conv2d_47 (Conv2D)          (32, 256, 256, 32)        896       
                                                                 
 max_pooling2d_47 (MaxPoolin  (32, 128, 128, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_48 (Conv2D)          (32, 128, 128, 64)        18496     
                                                                 
 max_pooling2d_48 (MaxPoolin  (32, 64, 64, 64)         0         
 g2D)                                                

In [75]:
tomato_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [59]:
steps_per_epoch = len(train_ds)//32

validation_steps = len(test_ds)//32 

In [76]:
try:
    tomato_model.fit(
        train_ds,
        epochs = 20,
        batch_size = 32,
        verbose = 1,
        validation_data= val_ds,
#         steps_per_epoch = steps_per_epoch,
#         validation_steps = validation_steps,
    )
except:
    pass

Epoch 1/20
110/110 [==============================] - 128s 1s/step - loss: 1.0272 - accuracy: 0.4584 - val_loss: 0.8881 - val_accuracy: 0.5521
Epoch 2/20
110/110 [==============================] - 124s 1s/step - loss: 0.8242 - accuracy: 0.6012 - val_loss: 0.6332 - val_accuracy: 0.7083
Epoch 3/20
110/110 [==============================] - 123s 1s/step - loss: 0.6341 - accuracy: 0.6904 - val_loss: 0.6317 - val_accuracy: 0.7063
Epoch 4/20
110/110 [==============================] - 123s 1s/step - loss: 0.5789 - accuracy: 0.7104 - val_loss: 0.5292 - val_accuracy: 0.7500
Epoch 5/20
110/110 [==============================] - 123s 1s/step - loss: 0.4650 - accuracy: 0.7600 - val_loss: 0.4097 - val_accuracy: 0.7896
Epoch 6/20
110/110 [==============================] - 123s 1s/step - loss: 0.3842 - accuracy: 0.8230 - val_loss: 0.3181 - val_accuracy: 0.8375
Epoch 7/20
110/110 [==============================] - 122s 1s/step - loss: 0.3218 - accuracy: 0.8558 - val_loss: 0.5582 - val_accuracy: 0.8021

In [77]:
try:
    tomato_model.evaluate(test_ds)
except:
    pass

16/16 [==============================] - 5s 190ms/step - loss: 0.3115 - accuracy: 0.8620


In [78]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)
    
    predictions = model.predict(img_array)
    
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100*(np.max(predictions[0])), 2)
    return predicted_class, confidence

In [79]:
tomato_model.save(f"../Models/tomato2")

INFO:tensorflow:Assets written to: ../Models/tomato2\assets


INFO:tensorflow:Assets written to: ../Models/tomato2\assets
